这是hot100的第93题：最长回文子串

In [ ]:
class Solution:
    def longestPalindrome(self, s: str) -> str:
        ans = s[0]
        l = 1
        dp = [[0 for i in s] for j in s]
        for i in range(len(s)):
           dp[i][i] = 1
        for i in range(len(s)//2,len(s)):
            for j in range(i,len(s)-i-2,-1):
                if i-j == 1 and s[i] == s[j]:
                    dp[i][j] = 2
                elif i-j >1 and s[i] == s[j] and dp[i-1][j+1] != 0:
                    dp[i][j] = dp[i-1][j+1]+2
                if l < dp[i][j]:
                    l = dp[i][j]
                    ans = s[j:i+1]
        return ans

这个做不了，因为是只从中间开始，没有照顾到各个点位扩散的情况

In [ ]:
class Solution:
    def longestPalindrome(self, s: str) -> str:
        ans = s[0]
        l = 1
        dp = [[0 for i in s] for j in s]
        for i in range(len(s)):
           dp[i][i] = 1
        for i in range(1,len(s)-1):
            m = min(i,len(s)-1-i)
            for j in range(1,m):
                if s[i-j] != s[i+j]:
                    break
                elif s[i-j] == s[i+j]:
                    dp[i-j][i+j] = dp[i-j+1][i+j-1]+2
                if 2*j+1 > l:
                    l = 2*j+1
                    ans  = s[i-j:i+j+1]
        for i in range(len(s)-1):
            if s[i] == s[i+1]:
                m = min(i,len(s)-i-2)
                for j in range(m):
                    if s[i-j] == s[i+1+j]:
                        dp[i-j][i+1+j] = dp[i-j+1][i+j] + 2
                    else:
                        break
                    if 2*j+2 > l:
                        l = 2*j+1
                        ans  = s[i-j:i+j+1]
            else:
                break
        return ans

In [ ]:
class Solution:
    def longestPalindrome(self, s: str) -> str:
        ans = s[0]
        l = 1
        for i in range(len(s)):
            left,right = i,i
            while left >= 0 and right <= len(s)-1 and s[left] == s[right]:
                left -= 1
                right += 1
            if right-left+1 > l:
                l = right-left+1
                ans = s[left+1:right]
            
            if i < len(s)-1:
                left,right = i,i+1
                while left >= 0 and right <= len(s)-1 and s[left] == s[right]:
                    left -= 1
                    right += 1
                if right-left+1 > l:
                    l = right-left+1
                    ans = s[left+1:right]
        return ans

第一个是我差不多独立想出来的，又像dp，又像从中间扩散，其实用从中间扩散的方法更好
第二个思路是将回文串分成了奇数和偶数可能，奇数时从i开始扩散，这里用for循环判断碰到边界不方便，用while循环的更好，判断left right是不是到了边界，终止条件就是当扩散到两边不一样，或者到了临界值，注意用while的话，最后的left和right都超过了一个，所以left+1，而因为切片所以right不变
最后判断大小长度更长的就返回ans了 

In [ ]:
class Solution:
    def longestPalindrome(self, s: str) -> str:
        dp = [[False]*len(s) for _ in s]
        start = 0
        l = 1
        for i in range(len(s)):
            dp[i][i] = 1
        for i in range(2,len(s)+1):
            for j in range(len(s)-i+1):
                k = i+j-1
                if s[j] == s[k]:
                    if j == k:
                        dp[i][j] = True
                    else:
                        dp[i][j] = dp[i+1][j-1]
                if dp[i][j] == True and j-i+1 > l:
                    start = i
                    l = j-i+1
        return s[start:start+l]

这个是dp的方法，dp[i][j]代表ij串是不是回文串，思路是用长度进行外层循环，因为这样才能有dp的过程，能够从外层向内部dp

In [ ]:
class Solution:
    def longestPalindrome(self, s: str) -> str:
        t = '#' + '#'.join(s) + '#'
        p = [0 for i in t]
        l = m = r = c = 0
        for i in range(len(t)):
            if i < r:
                mirror = 2*c - i
                p[i] = min(r-i,p[mirror])
            left,right = i-(p[i]+1),i+(p[i]+1)
            while left >= 0 and right <= len(t)-1 and t[left] == t[right]:
                left -= 1
                right += 1
                p[i] += 1
            if p[i] + i > c:
                r = p[i] + i
                c = i
            if p[i] > l:
                l = p[i]
                m = i
        i = (m-l)//2
        return s[i:i+l]

这个是叫manacher方法，思路是减少不必要的扩散，只做必要的扩散，也就是因为当i处在已知的边界内时，根据回文串的特点，它在边界所对应的回文串中心有一个对称点，那么我就可以拿到那个点的对称半径，与边界r减去当前点位i的取最小值，也就用着省去不必要的扩散
但是这个只是省去了到边界的扩散，还要继续扩散，得到新的边界，与先前的边界对比，如果大了，就可以更新最大边界和中心，
p[i]是半径，当最大半径超过l时，说明找到了更大的，就可以更新了
最后把中心减去半径，整除二就是原始字符串的最大回文串索引，半径就是长度范围
时间复杂度就是o(N)